# Загрузка Pandas и очистка данных

In [1]:
import pandas as pd
import re
from datetime import datetime

In [2]:
df = pd.read_csv('main_task.xls')

In [3]:
#Смотрим на типы данных в стобцах
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Restaurant_id      40000 non-null  object 
 1   City               40000 non-null  object 
 2   Cuisine Style      30717 non-null  object 
 3   Ranking            40000 non-null  float64
 4   Rating             40000 non-null  float64
 5   Price Range        26114 non-null  object 
 6   Number of Reviews  37457 non-null  float64
 7   Reviews            40000 non-null  object 
 8   URL_TA             40000 non-null  object 
 9   ID_TA              40000 non-null  object 
dtypes: float64(3), object(7)
memory usage: 3.1+ MB


Колонки Cuisine Style,Price Range и Number of Reviews содержат пропуски в данных - их нужно будет заполнить либо удалить сам стоблец.

### Уровень цен (Price Range)

In [4]:
df['Price Range'].unique()

array(['$$ - $$$', nan, '$$$$', '$'], dtype=object)

In [5]:
len(df[df['Price Range']=='$$ - $$$'])

18412

Столбец содержит значения в формате символов - их нужно перевести в числовой формат:

In [6]:
def convert_prices_to_numeric(x):
    if x=='$':
        return 1
    elif x=='$$ - $$$':
        return 2
    elif x=='$$$$':
        return 3

In [7]:
df['Price']=df['Price Range'].apply(convert_prices_to_numeric)

Теперь попробуем заполнить пропуски в данных средним значением по столбцу:

In [8]:
df['Price'].fillna(df['Price'].mean())

0        2.000000
1        1.814046
2        3.000000
3        1.814046
4        2.000000
           ...   
39995    2.000000
39996    2.000000
39997    1.814046
39998    2.000000
39999    2.000000
Name: Price, Length: 40000, dtype: float64

Среднее значение получилось дробным, что не позволяет отнести его к одному из трех уровней цен - поэтому используем медиану вместо среднего.

In [9]:
df['Price']=df['Price'].fillna(df['Price'].median())

Теперь данные об уровне цен в ресторанах имеют числовые значения и могут быть добавлены в качестве параметра:

In [10]:
df['Price'].unique()

array([2., 3., 1.])

### Города (City)

In [11]:
df['City'].unique()

array(['Paris', 'Stockholm', 'London', 'Berlin', 'Munich', 'Oporto',
       'Milan', 'Bratislava', 'Vienna', 'Rome', 'Barcelona', 'Madrid',
       'Dublin', 'Brussels', 'Zurich', 'Warsaw', 'Budapest', 'Copenhagen',
       'Amsterdam', 'Lyon', 'Hamburg', 'Lisbon', 'Prague', 'Oslo',
       'Helsinki', 'Edinburgh', 'Geneva', 'Ljubljana', 'Athens',
       'Luxembourg', 'Krakow'], dtype=object)

In [12]:
df['City'].nunique()

31

У нас есть данные о ресторанах в 31 городе - при этому пропуски отсутствуют.

Для того чтобы добавить признак города в выборку создадим отдельный dataframe, в котором каждый столбец будет соответствовать одному городу из списка со значениями 0 или 1 в зависимости от того, относится наблюдение к этому городу или нет.

In [13]:
cities=pd.DataFrame()
for city in df['City'].unique():
    cities[city]=(df['City']==city).astype(int)

In [14]:
cities.head()

,Paris,Stockholm,London,Berlin,Munich,Oporto,Milan,Bratislava,Vienna,Rome,...,Lisbon,Prague,Oslo,Helsinki,Edinburgh,Geneva,Ljubljana,Athens,Luxembourg,Krakow
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Добавим в новый датафрейм колонку с рейтингом:

In [15]:
cities['Rating']=df['Rating']

Затем построим матрицу корреляции и опытным путем выясним, для каких городов коэффициент корреляции с рейтингом окажется больше, чем для остальных - именно сведения о них стоит включить в выборку.

In [16]:
#с помощью iloc[:-1] убираем коэффициент корреляции rating с самим собой
#nlargest() возвращает n-максимальных значений в ряду данных
most_important_cities=cities.corr()['Rating'].iloc[:-1].nlargest(5).index.values.tolist() 

In [17]:
df['Important_cities']=df['City'].apply(lambda x: 1 if x in most_important_cities else 0)

In [18]:
least_important_cities=cities.corr()['Rating'].iloc[:-1].nsmallest(3).index.values.tolist() 

In [19]:
df['Less_important_cities']=df['City'].apply(lambda x: 1 if x in least_important_cities else 0)

Помимо этого мы можем выделить топ-5 городов, которые в целом наиболее часто встречаются в выборке, и также использовать эту информацию для построения модели.

In [ ]:
df['Popular_cities']=df['City'].apply(lambda x: 1 if x in df['City'].value_counts().head(5) else 0)

### Кухня (Cuisine Style)

In [ ]:
df['Cuisine Style'].unique()

Строки столбца представляют собой скрытые списки + присутствуют пропущенные значения (None) - поэтому для извлечения оттуда информации сначала заполним пропуски и удалим лишние скобы и кавычки.

*Так как в столбце содержатся данные о видах кухни, то в данном случае имеет смысл заполнить пропуски придуманной категорией "Other" ('Другая'):*

In [ ]:
df['Cuisine Style']=df['Cuisine Style'].fillna('Other')

In [ ]:
df['Cuisine Style']=df['Cuisine Style'].str.strip("[]").str.replace("'","")

Теперь посчитаем общее число уникальных значений в столбце (разновидностей кухни):

In [ ]:
#Не забываем, что одну из категорий ("Other") мы добавили сами
list_of_cuisine=pd.DataFrame(df['Cuisine Style'].astype(str).str.split(',').tolist()).stack().str.strip()
list_of_cuisine.nunique()

Определим наиболее популярную разновидность кухни, используя метод value_counts:

In [ ]:
list_of_cuisine.value_counts()

Теперь посчитаем число разновидностей кухонь для каждого ресторана и найдем, какое число разных кухонь в среднем предлагает каждое заведение:

In [ ]:
number_of_cuisine=list_of_cuisine.reset_index().groupby(['level_0'])[0].count()
round(number_of_cuisine.mean(),1)

Добавим в данные столбец, отражающий количество различных кухонь в одном заведения:

In [ ]:
df['Number of cuisines']=list_of_cuisine.reset_index().groupby(['level_0'])[0].count()

Также сейчас в мире активно набирает популярность вегетарианская кухня (и вегетарианство как идея в целом), поэтому можно предположить, что наличие такой опции будет для ресторана значимым преимуществом - добавим столбец, обозначающий наличие/отсутствие вегетарианских блюд в меню ресторана (1/0):

In [ ]:
df['Vegetarian Friendly']=df['Cuisine Style'].apply(lambda x: 1 if 'Vegetarian Friendly' in x else 0)

### Отзывы (Reviews)

Используем регулярные выражения, чтобы вытащить из данных информацию о дате последнего отзыва (по каждому ресторану):

In [ ]:
def last_review(row):
    pd.DataFrame()
    pattern = re.compile('\d+[/]\d+[/]\d+')
    list_of_date=pattern.findall(row)
    if not list_of_date:
        return None
    else:
        list_of_date=[pd.to_datetime(date) for date in list_of_date]
        return max(list_of_date)

In [ ]:
recent_review=df['Reviews'].apply(last_review)

Затем находим самый свежий отзыв на сайте в целом:

In [ ]:
most_recent_review=recent_review.max()
most_recent_review

Теперь, используя эти данные, можно посчитать количество дней между самым свежим отзывом для каждого ресторана и самым свежим отзывом на сайте (то есть понять, насколько давно публиковался последний отзыв для того или иного ресторана):

In [ ]:
df['Last_review_days_ago']=most_recent_review-recent_review

In [ ]:
df['Last_review_days_ago']=df['Last_review_days_ago'].apply(lambda x: x.days)

*Осталось заполнить пропуски в новых столбцах:*

Пропуски в столбце с количеством отзывов можно заполнить средним значением по стобцу:

In [ ]:
df['Number of Reviews']=df['Number of Reviews'].fillna(df['Number of Reviews'].mean())

Так как в столбце last_review_date содержатся данные о количестве дней, прошедших с даты публикации последнего отзыва о ресторане до даты публикации самого свежего отзыва на сайте, то пропущенные значения, которые обозначают отсутствие отзывов о ресторане, можно заполнить максимальным значением по столбцу, то есть считать их наиболее устаревшими:

In [ ]:
df['Last_review_days_ago']=df['Last_review_days_ago'].fillna(df['Last_review_days_ago'].max())

### Дополнительная обработка

In [ ]:
#Удаляем нечисловые столбцы
df.drop(columns=['Restaurant_id','City','Cuisine Style','Price Range','Reviews','URL_TA','ID_TA'], inplace=True)

In [ ]:
#Смотрим, остались ли в каких-то столбцах пропущенные значения
df.isna().any()

Теперь данные обработаны и готовы для построения модели.

# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели

In [ ]:
# Х - данные с информацией о ресторанах, у - целевая переменная (рейтинги ресторанов)
X = df.drop(['Rating'], axis = 1)
y = df['Rating']

In [ ]:
# Загружаем специальный инструмент для разбивки:
from sklearn.model_selection import train_test_split

In [ ]:
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.
# Для тестирования мы будем использовать 25% от исходного датасета.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

# Создаём, обучаем и тестируем модель

In [ ]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [ ]:
# Создаём модель
regr = RandomForestRegressor(n_estimators=100)

# Обучаем модель на тестовом наборе данных
regr.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = regr.predict(X_test)

In [ ]:
y_pred

In [ ]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

In [ ]:
df.corr()

#### Итак, с помощью обработки данных и создания новых фичей нам удалось понизить среднюю абсолютную ошибку с 0,42 до 0,30 и построить более качественную модель, предсказывающую реальные оценки ресторанов на сервисе TripAdvisor.

In [ ]:
result=pd.DataFrame(y_pred).reset_index()
result.columns=['Restaurant_id','Rating']
result['Restaurant_id']='id_'+result['Restaurant_id'].astype(str)
result.set_index('Restaurant_id', drop=True, inplace=True)
result.to_csv('submission.csv')